In [8]:
import os

In [9]:
%pwd

'c:\\Users\\ahmed\\OneDrive\\Documents\\Documents\\coding\\data science\\DataScience\\MLOPS-Project\\research'

In [10]:
os.chdir("../")

In [11]:
%pwd

'c:\\Users\\ahmed\\OneDrive\\Documents\\Documents\\coding\\data science\\DataScience\\MLOPS-Project'

## entity

In [12]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen= True) 
class DataTransformationConfig: # defining the schema for configuration manager
    root_dir: Path
    data_path: Path 

## Update config

In [13]:
from mlproject.constants import * 
from mlproject.utils.common import read_yaml, create_directories

class ConfigurationManager():
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path
        )

        return data_transformation_config


In [7]:
import os 
from mlproject import logger 
from sklearn.model_selection import train_test_split
import pandas as pd 
from sklearn.preprocessing import LabelEncoder



In [14]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

#     def transforming(self):
#         df = pd.read_csv(self.config.data_path)
#         df = df.drop(columns=['customerID'])
#         df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# # label encoding
#         le = LabelEncoder()
#         columns_to_encode = ['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']

#         for col in columns_to_encode:
#             df[col] = le.fit_transform(df[col])

# # one hot encoding

#         columns_to_onehot = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']

#         for col in columns_to_onehot:
 
#             df = pd.get_dummies(df, columns=[col], prefix=col, drop_first=False)

# # ensuring all data is numerical 
#         for col in df.columns:
#             if df[col].dtype == bool:
#                 df[col] = df[col].astype(int)

#         df.to_csv(os.path.join(self.config.root_dir, 'transformed.csv'), index= False)
        
#################################################################################################################################################

    def preprocessing(self):

        df = pd.read_csv(self.config.data_path)
        df = df.drop(columns=['customerID'])
        df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
        df.dropna(subset=['TotalCharges'], inplace=True)

# label encoding
        le = LabelEncoder()
        columns_to_encode = ['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']

        for col in columns_to_encode:
            df[col] = le.fit_transform(df[col])

# one hot encoding

        columns_to_onehot = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']

        for col in columns_to_onehot:
 
            df = pd.get_dummies(df, columns=[col], prefix=col, drop_first=False)

# ensuring all data is numerical 
        for col in df.columns:
            if df[col].dtype == bool:
                df[col] = df[col].astype(int)

        train, test = train_test_split(df)

        train.to_csv(os.path.join(self.config.root_dir, 'train.csv'), index = False)
        test.to_csv(os.path.join(self.config.root_dir, 'test.csv'), index = False)

        logger.info('Splitted the data into traininf and test sets')
        logger.info('train.shape')
        logger.info('test.shape')

        print(train.shape)
        print(test.shape)

## pipeline

In [9]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.preprocessing()
except Exception as e:
    raise e 

[2025-09-09 07:54:06,988: INFO: common: yaml file: config\config.yaml, loaded successfully]
[2025-09-09 07:54:06,992: INFO: common: yaml file: params.yaml, loaded successfully]
[2025-09-09 07:54:06,995: INFO: common: yaml file: schema.yaml, loaded successfully]
[2025-09-09 07:54:06,995: INFO: common: created directory at: artifacts]
[2025-09-09 07:54:06,997: INFO: common: created directory at: artifacts/data_transformation]
[2025-09-09 07:54:07,246: INFO: 3088094016: Splitted the data into traininf and test sets]
[2025-09-09 07:54:07,247: INFO: 3088094016: train.shape]
[2025-09-09 07:54:07,247: INFO: 3088094016: test.shape]
(5282, 41)
(1761, 41)
